In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
QCL_df = pd.read_csv('data/faostat/QCL_clean.csv')
RL_df = pd.read_csv('data/faostat/RL_clean.csv')

In [3]:
QCL_df.drop(columns=['Unit'],inplace=True)

In [4]:
QCL_df.dtypes

Area     object
Item     object
Year      int64
Value     int64
dtype: object

In [5]:
QCL_df.Item.unique()

array(['Rice', 'Cattle and Buffaloes', 'Soya beans', 'Cocoa beans',
       'Coffee, green', 'Oil palm fruit', 'Green corn/Maize'],
      dtype=object)

In [6]:
QCL_country = QCL_df.groupby('Area').agg(list)

In [7]:
QCLdict = {}
for name in QCL_country.index:
    dfx = pd.DataFrame([*QCL_country.loc[name]], index=['Name','Year','Value']).T
    dfx = dfx.groupby('Name').agg(list)
    items=[]
    for n in dfx.index:
        unit = 'hectare'
        if n=='Cattle and Buffaloes': unit = 'animal'
        items.append({'item':n,'unit':unit,'values':[*dfx.Value[n]],'years':[*dfx.Year[n]]})
    QCLdict[name] = items

In [8]:
QCLjson = pd.DataFrame({'name': QCLdict.keys(), 'items':QCLdict.values()})
QCLjson.set_index('name',inplace=True)
QCLjson

,items
name,
Afghanistan,"[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."
Albania,"[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."
Algeria,"[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."
Angola,"[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."
Antigua and Barbuda,"[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."
...,...
Venezuela (Bolivarian Republic of),"[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."
Viet Nam,"[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."
Yemen,"[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."


In [9]:
QCLjson.T.to_json('data/faostat/qcl.json')

In [10]:
RL_df.dtypes

Area                               object
Year                                int64
Land area                         float64
Cropland                          float64
Permanent meadows and pastures    float64
Forest land                       float64
Other land                        float64
Farm buildings and Farmyards      float64
Unaccounted                       float64
dtype: object

In [11]:
RL_country = RL_df.groupby('Area').agg(list)

In [12]:
area = [a[0] for a in RL_country['Land area']]

In [13]:
landuse = ['Cropland','Permanent meadows and pastures',
           'Forest land','Other land', 'Farm buildings and Farmyards','Unaccounted']

In [14]:
RLdict = {}
for name in RL_country.index:
    usage = []
    for use in landuse:
        if sum([*RL_country.loc[name,use]]) != 0:
            usage.append({'type':use,'unit':'1000 hectare',
                          'values':[*RL_country.loc[name,use]],'years':[*RL_country.loc[name,'Year']]})
    RLdict[name] = usage

In [15]:
RLjson = pd.DataFrame({'name': RLdict.keys(), 'landarea':area,'landuse':RLdict.values()})
RLjson.set_index('name',inplace=True)
RLjson

,landarea,landuse
name,,
Afghanistan,65223.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ..."
Albania,2740.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ..."
Algeria,238174.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ..."
American Samoa,20.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ..."
Andorra,47.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ..."
...,...,...
Wallis and Futuna Islands,14.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ..."
Western Sahara,26600.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ..."
Yemen,52797.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ..."


In [16]:
RLjson.T.to_json('data/faostat/rl.json')

In [17]:
FAOjson = RLjson.join(QCLjson)

In [18]:
FAOjson

,landarea,landuse,items
name,,,
Afghanistan,65223.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ...","[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."
Albania,2740.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ...","[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."
Algeria,238174.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ...","[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."
American Samoa,20.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ...",NaN
Andorra,47.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ...",NaN
...,...,...,...
Wallis and Futuna Islands,14.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ...",NaN
Western Sahara,26600.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ...",NaN
Yemen,52797.0,"[{'type': 'Cropland', 'unit': '1000 hectare', ...","[{'item': 'Cattle and Buffaloes', 'unit': 'ani..."


In [19]:
FAOjson.T.to_json('data/faostat/faostat.json')